# ニューストピック分類 前処理


## Google ドライブの接続  
以下を実行。表示されるurlからGoogleアカウントにサイン。表示される認証用のコードを以下に表示されるテキストボックスにコピペ。  
→ サインインしたGoogleアカウントのGoogleドライブがマウントされる。

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## 主な設定

In [2]:
data_path = '/content/drive/My Drive/Colab Notebooks/data'   # 作業用フォルダ

## データセットの読み込み
ロンウイット社が公開している[livedoorニュースのデータセット](https://www.rondhuit.com/download.html#ldcc)を読み込む。

In [3]:
import os
dl_file_path = os.path.join(data_path, 'ldcc-20140209.tar.gz')

In [4]:
! wget -c -P '{data_path}' https://www.rondhuit.com/download/ldcc-20140209.tar.gz
! tar -zxf '{dl_file_path}' -C '{data_path}'

--2021-09-05 22:42:42--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [5]:
import glob
import pandas as pd
text_data  = []   # 文章のリスト
label_data = []   # ラベルのリスト
cat_counts = []   # カテゴリ別サンプル数
text_path = os.path.join(data_path, "text")
dir_names = [base_name for base_name in os.listdir(path=text_path)
    if os.path.isdir(os.path.join(text_path, base_name))]  # ディレクトリ一覧
for i, dir_name in enumerate(dir_names):
    files = glob.glob(os.path.join(text_path, dir_name, "*.txt"))  # ファイル一覧
    for j, file in enumerate(files):
        if os.path.basename(file) == "LICENSE.txt":
            continue
        with open(file, "r") as f:
            text = f.readlines()[3:]  # 記事を読み込み、見出しを削除する
        text = "".join(text)
        text = text.translate(
            str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))
            # 簡易的なクレンジング
        text_data.append(text)
        label_data.append(i)
    cat_counts.append(j+1)
text_label_data = pd.DataFrame({'text':text_data, 'label':label_data})
cat_info = pd.DataFrame({
    'label':range(len(dir_names)), 
    'name':dir_names,
    'cat_count': cat_counts})
cat_info

,label,name,cat_count
0,0,dokujo-tsushin,871
1,1,livedoor-homme,512
2,2,kaden-channel,865
3,3,smax,871
4,4,topic-news,771
5,5,peachy,843
6,6,movie-enter,871
7,7,it-life-hack,871
8,8,sports-watch,901


## データセットの分割と確認

In [6]:
from sklearn.model_selection import train_test_split
news_train, news_test = train_test_split(text_label_data, shuffle=True, 
            stratify=text_label_data['label'])  # 訓練用とテスト用に分割
news_train.head()

,text,label
3956,「クリスマスだから、どこかリゾートに行きたい」と思いつつも、師走の忙しい最中に時間を作るのは...,5
521,節電モードの冬。各地のクリスマスイルミネーションも縮小気味らしい。そんな冬だからこそ、今年は...,0
4558,世界屈指のクリエイターたちが丁寧に紡いだ『ももへの手紙』は、豊かな自然とやさしい人々が生きる...,5
2541,いよいよ本格展開の「IPv6」がスマートフォンに投げかける課題とは？ 6/6から「World...,3
4065,2002年の誕生から、今回で15回目となる「神戸コレクション2010 SPRING/SUMM...,5


In [7]:
news_test.head()

,text,label
4540,2008年、”鉄の女”のニックネームを持つ元英国首相マーガレット・サッチャーが、実は認知症に...,5
3341,箱根駅伝に出場し、4区を走った東京農業大学・内藤寛人選手の部屋の様子が日本テレビ系情報番組「...,4
4516,キヤノンが女子に大人気の小物ブランド「Samantha Thavasa Petit Choi...,5
4440,アレクサンドル・デュマの名作文学『三銃士』が、現代の壮大なエンタテインメント作品として生まれ...,5
5691,悪質なコンピュータウイルスが氾濫しているWindowsに対して「Macは安全」とされる。Ma...,7


## データの保存

In [8]:

news_train.to_csv(os.path.join(data_path, "news_train.csv"), index=False)
news_test .to_csv(os.path.join(data_path, "news_test.csv" ), index=False)
cat_info  .to_csv(os.path.join(data_path, "cat_info.csv"  ), index=False)

## データのロード確認

In [9]:
news_train = pd.read_csv(
    os.path.join(data_path, "news_train.csv"), index_col=False)
news_train.head()

,text,label
0,「クリスマスだから、どこかリゾートに行きたい」と思いつつも、師走の忙しい最中に時間を作るのは...,5
1,節電モードの冬。各地のクリスマスイルミネーションも縮小気味らしい。そんな冬だからこそ、今年は...,0
2,世界屈指のクリエイターたちが丁寧に紡いだ『ももへの手紙』は、豊かな自然とやさしい人々が生きる...,5
3,いよいよ本格展開の「IPv6」がスマートフォンに投げかける課題とは？ 6/6から「World...,3
4,2002年の誕生から、今回で15回目となる「神戸コレクション2010 SPRING/SUMM...,5


In [10]:
news_test = pd.read_csv(
    os.path.join(data_path, "news_test.csv" ), index_col=False)
news_test.head()

,text,label
0,2008年、”鉄の女”のニックネームを持つ元英国首相マーガレット・サッチャーが、実は認知症に...,5
1,箱根駅伝に出場し、4区を走った東京農業大学・内藤寛人選手の部屋の様子が日本テレビ系情報番組「...,4
2,キヤノンが女子に大人気の小物ブランド「Samantha Thavasa Petit Choi...,5
3,アレクサンドル・デュマの名作文学『三銃士』が、現代の壮大なエンタテインメント作品として生まれ...,5
4,悪質なコンピュータウイルスが氾濫しているWindowsに対して「Macは安全」とされる。Ma...,7


In [11]:
cat_info = pd.read_csv(
    os.path.join(data_path, "cat_info.csv" ), index_col=False)
cat_info

,label,name,cat_count
0,0,dokujo-tsushin,871
1,1,livedoor-homme,512
2,2,kaden-channel,865
3,3,smax,871
4,4,topic-news,771
5,5,peachy,843
6,6,movie-enter,871
7,7,it-life-hack,871
8,8,sports-watch,901
